In [6]:
using MatrixNetworks
using Distributions
using Gadfly
using Clustering

adj = readdlm("Data/Facebook/facebook-gender_edges.txt");
lab = readdlm("Data/Facebook/facebook-gender_labels.txt");

adj = adj+ones(size(adj));

community_true0 = lab[:,2]+1;
lab_gender = 0;
N = length(community_true0);
n_edge0 = size(adj)[1];

A0 = sparse(adj[:,1],adj[:,2],vec(ones(Int64,n_edge0,1)),N,N);

lab_gender = 0;
adj = 0;
gc();

#Assortative
#A0 = (A0+A0');
A,p = largest_component(A0);

community_region =community_true0[p];

N = size(A)[1];
n_edge0 = nnz(A)/2;

d = sum(A,2);
s = sum(d);

In [7]:
r = 80;
N = length(d); 

v0 = rand(Uniform(-1,1),N,r);
v0 = v0./sqrt.(sum(abs2, v0, 1));
v0, t = qr(v0);
l0 = diag(t);


### Preprocess ###
u0 = rand(Uniform(-1,1),N,1);
u0 = u0./norm(u0);
n=0;
for it = 1:30
    u0 = A*u0 .- (d/s)*(d'*u0);
    n = norm(u0,2);
    u0 = u0/n;
end
shift = n;

l = l0;
##################

it_max=10000;
it = 1;
while it <=it_max
    v0 = A*v0 .- (d/s)*(d'*v0) .+ shift*v0;
    v0, t = qr(v0);
    l = diag(t);
    if norm(vec(abs.(l))-vec(abs.(l0)))<1e-03
             break;
    end
    l0 = l;
    it = it+1;
end

if it ==it_max
        print("No convergence")
end 


In [7]:
print(l0)

[298.452, 262.594, 241.339, 218.548, 214.46, 207.119, 199.034, 194.876, 194.321, 192.341, 190.173, 188.528, 187.581, 187.066, 184.681, 184.231, 183.431, 181.318, 179.693, 179.227, 179.251, 176.902, 176.453, 175.506, 174.405, 173.74, 173.216, 173.321, 172.782, 172.309, 172.657, 170.962, 170.961, 170.787, 170.5, 170.09, 169.157, 169.413, 168.919, 168.811, 168.379, 167.964, 168.256, 167.607, 167.378, 166.537, 165.917, 165.915, 165.917, 165.361, 165.321, 165.187, 164.943, 164.789, 165.051, 164.873, 164.817, 164.177, 164.227, 164.124, 163.963, 163.785, 163.689, 163.581, 163.513, 163.502, 163.631, 163.222, 163.21, 163.049, 163.001, 162.959, 162.656, 162.664, 162.61, 162.269, 162.07, 162.299, 162.034, 161.918]

In [19]:
include("Algo/Partition.jl")
include("Algo/UpdateCentroids.jl")
include("Algo/FindCommunity.jl")
include("Algo/Reorder.jl")

Sig = v0[:,1:8]';

n_clusters = 200;
n_it_Kmeans = 1000;
n_updates = 100;


community = vec(zeros(N,1));
community0 = vec(zeros(N,1));
Q_best = 0;
n_c_best = 0;
n_c = 0;
# Probability mass for sampling the initial centroids
p = vec(d/s);
n_updates_best = 0;

for k =1:n_it_Kmeans

	community0, Q, n_updates = Partition(A,Sig,n_updates,n_clusters,p);
	n_c = length(unique(community0));

    	if Q>Q_best
		community = community0;
		Q_best = Q;
		n_c_best = n_c;
		n_updates_best = n_updates;

		
    	end
end	
  
	community = Reorder(community);

	print("Number of updates: ")
	print(n_updates_best)
	print("\n")
	print("Number of communities: ")
	print(n_c_best)
	print("\n")

	print("Modularity: ")
	print(Q_best)
	print("\n")

Number of updates: 1
Number of communities: 10
Modularity: 0.8198214628396684


In [8]:
include("Algo/UpdateCentroids.jl")
include("Algo/FindCommunity.jl")
include("Algo/Reorder.jl")
include("Algo/Partition.jl")

n_clusters = 200;
n_it_Kmeans = 500;
n_updates = 100;
community = vec(zeros(N,1));
community0 = vec(zeros(N,1));

Q_best_best = 0;
n_c_best_best = 0;
n_updates_best_best = 0;
r_best = 2;
for r=2:80
    
    Sig = v0[:,1:r]';
    Q_best = 0;
    n_c_best = 0;
    n_c = 0;
    # Probability mass for sampling the initial centroids
    p = vec(d/s);
    n_updates_best = 0;

    for k =1:n_it_Kmeans

        community0, Q, n_updates = Partition(A,Sig,n_updates,n_clusters,p);
        n_c = length(unique(community0));
            if Q>Q_best
            community = community0;
            Q_best = Q;
            n_c_best = n_c;
            n_updates_best = n_updates;
            end
    end	

    community = Reorder(community);
    
    if Q_best>Q_best_best 
        r_best = r;
        Q_best_best = Q_best;
        n_c_best_best = n_c_best;
        n_updates_best_best = n_updates_best;
    end
    
end


	print("Number of updates: ")
	print(n_updates_best_best )
	print("\n")
	print("Number of communities: ")
	print(n_c_best_best )
	print("\n")

	print("Modularity: ")
	print(Q_best_best )
	print("\n")
	print("n: ")
	print(r_best )
	print("\n")


Number of updates: 0
Number of communities: 13
Modularity: 0.7729408271342167
n: 5
